In [ ]:
import numpy as np
import cv2

PREVIEW = 0    
BLUR = 1       
FEATURES = 2   
CANNY = 3      
feature_params = dict(maxCorners=500,
                      qualityLevel=0.2,
                      minDistance=15,
                      blockSize=9)
image_filter = PREVIEW

source = cv2.VideoCapture(0) 

if not source.isOpened():
    print("Error: Cannot open the camera.")
    exit()


win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

while True:
    ret, frame = source.read()  
    if not ret:
        print("Error: Unable to read frame.")
        break

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == CANNY:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        result = cv2.Canny(gray_frame, 80, 150)      
    elif image_filter == BLUR:
        result = cv2.blur(frame, (13, 13))
    elif image_filter == FEATURES:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)

        result = frame.copy()
        if corners is not None:
            for x, y in np.float32(corners).reshape(-1, 2):
                cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key == ord('q') or key == ord('Q') or key == 27:
        break
    elif key == ord('c') or key == ord('C'): 
        image_filter = CANNY
    elif key == ord('b') or key == ord('B'):
        image_filter = BLUR
    elif key == ord('f') or key == ord('F'):
        image_filter = FEATURES
    elif key == ord('p') or key == ord('P'):
        image_filter = PREVIEW

source.release()
cv2.destroyAllWindows()
